# Word Embeddings for Text Classification
We start be loading the Word2Vec word embeddings

In [1]:
#To Do (Possibel that this cell should be deleted)
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

## Preprocessing data
Below we process the movie review data

In [2]:
from os import listdir
folder = "aclImdb/"
test_folder = "test/"
train_folder = "train/"
neg_folder = "neg/"
pos_folder = "pos/"
print("Getting Contents From Training Files")
path = folder + train_folder + pos_folder
pro_train  = [open(path + f, 'r').read().split() for f in listdir(path)]
path = folder + train_folder + neg_folder
neg_train  = [open(path + f, 'r').read().split() for f in listdir(path)]
print("Getting Contents From Test Files")
path = folder + test_folder + pos_folder
profile = listdir(path)
pro_test  = [open(path + f, 'r').read().split() for f in listdir(path)]
path = folder + test_folder + neg_folder
negfile = listdir(path)
neg_test  = [open(path + f, 'r').read().split() for f in listdir(path)]

Getting Contents From Training Files
Getting Contents From Test Files


## Embedding the Documents
We now need to get the average embeddings of each review so that we can train our model on a vector respresentation of the documents

In [3]:
import numpy as np
def averageEmbedding(review):
    return np.sum([model.get_vector(w) for w in review if w in model.vocab], axis = 0) / len(review)

def getAllEmbeddings(reviews):
    return [averageEmbedding(r) for r in reviews]

pro_train_embedded = getAllEmbeddings(pro_train)
pro_test_embedded = getAllEmbeddings(pro_test)
neg_train_embedded = getAllEmbeddings(neg_train)
neg_test_embedded = getAllEmbeddings(neg_test)


## Training A Classifier
Now that we have embedding for each review we can train a classifier.


In [5]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
Train = np.concatenate((pro_train_embedded, neg_train_embedded))
Test = np.concatenate((pro_test_embedded, neg_test_embedded))
train_labels = [0 for i in range(12500)] + [1 for i in range(12500)]
test_labels = [0 for i in range(12500)] + [1 for i in range(12500)]
classifiers = {'random_forest': RandomForestClassifier(),
               'sgd': SGDClassifier(loss='hinge')}

for name, model in classifiers.items():
    print("Training " + name)
    model.fit(Train, train_labels)
    predictions = model.predict(Test)
    print(name, f1_score(predictions, test_labels, average="micro"))
    print(name, f1_score(predictions, test_labels, average="macro"))


Training random_forest
random_forest 0.72
random_forest 0.7185278493113783
Training sgd
sgd 0.81916
sgd 0.8191341286563776


## Conclusion
We get a Test Accuracy of  **0.8152** when using linear SVMs for classifying sentiments of reviews using their average word2vec Embedding.